# Blogs related to this exercise:

[Eigenface using OpenCV](https://www.learnopencv.com/eigenface-using-opencv-c-python/)

In [ ]:
# Problems to be solved: 
# How to align images if not done in the dataset
# How eigen face are used in face recognition & landmark detection

"""
As shown in the video:

 - The first principal component of these face images is actually the image background
 - The second principal component of these face images is actually the hair style
 - The third principal component corresponds to pose, lighting and hairs
 - The other principal components including male/female, with or without glasses, the coolness or warmness of the image ...
 
The great thing about this eigen face implementation is that it gives me a concrete example of understanding PCA and another
perspective to look at face images.
"""
import os
import sys
import cv2
import numpy as np

# Create data matrix for all the images
def create_data_matrix(images):
    print("Creating data matrix", end="...")
    
    num_images = len(images)
    size = images[0].shape
    
    # Allocate space for a data matrix
    data = np.zeros((num_images, size[0] * size[1] * size[2]), dtype=np.float32)
    for i in range(0, num_images):
        image = images[i].flatten()
        data[i, :] = image
    
    print("DONE!")
    return data

# Read image from the directory
def read_images(path):
    print("Reading images from " + path + "...")
    
    images = []
    # List all files in the directory and read data from text files one by one
    for file_path in sorted(os.listdir(path)):
        file_extension = os.path.splitext(file_path)[1] 
        if file_extension in [".jpg", ".jpeg"]:
            image_path = os.path.join(path, file_path)
            image = cv2.imread(image_path)
        
        if image is None:
            print("Image:{} not read properly".format(image_path))
        else:
            # Convert image to floating point
            image = np.float32(image) / 255.0
            # Add image to list
            images.append(image)
            # Flip image
            image_flip = cv2.flip(image, 1)
            images.append(image_flip)
    
    num_images = len(images) // 2
    
    if num_images == 0:
        print("No images found")
        sys.exit(0)
        
    print(str(num_images) + " files read.")
    
    return images

# Add the weighted eigen faces to the mean/average face
def create_new_face(*args):
    # Start with the mean image
    output = average_face
    
    # Add the eigen faces with weights, faces are generated by `F = average_face + weights[i] * eigen_faces[i]`
    for i in range(face_components):
        # As OpenCV does not allow slide values to be negative
        # Here we use weight = tracker_value - max_tracker_value // 2
        tracker_values[i] = cv2.getTrackbarPos("Weight" + str(i), "Trackbars")
        weight = tracker_values[i] - max_tracker_value // 2
        output = np.add(output, eigen_faces[i] * weight)
        
    output = cv2.resize(output, (0, 0), fx=2, fy=2)
    cv2.imshow("Result", output)
    
def reset_tracker_values(*args):
    for x in range(0, face_components):
        cv2.setTrackbarPos("Weight" + str(x), "Trackbars", max_tracker_value // 2)
    create_new_face()
    
# Number of principal components to be kept
face_components = 10

# Maximum weights range
max_tracker_value = 255

# Images directory
image_path = "images/celeba_faces"

# Read images
images = read_images(image_path)

# Size of images
size = images[0].shape
print("The size of the image is", size)
# The size of the image is (218, 178, 3)

# Create a data matrix for PCA
data = create_data_matrix(images)
print("The shape of the data matrix is", data.shape)

# Compute the eigenvectors from the stack of images created
print("Calculating PCA ...")
mean, eigen_vectors = cv2.PCACompute(data, mean=None, maxComponents=face_components)
print("The shape of the eigen_vectors is {}".format(eigen_vectors.shape))
# The shape of the eigen_vectors is (10, 116412)
print("PCA compute is done!")

# The average_face is just the face constructed with the averaged vector of all face images
average_face = mean.reshape(size)

# eigen faces are faces constructed with eigen_vectors
eigen_faces = []
for eigen_vector in eigen_vectors:
    eigen_face = eigen_vector.reshape(size)
    eigen_faces.append(eigen_face)

# Create a window for displaying average face
cv2.namedWindow("Result", cv2.WINDOW_AUTOSIZE)

# Display results at 2x size
output = cv2.resize(average_face, (0, 0), fx=2, fy=2)
print("The shape of the output", output.shape)
cv2.imshow("Result", output)

# Create a window for displaying trackbars
cv2.namedWindow("Trackbars", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Trackbars", 356, 436)

# Values for controlling the trackbars
tracker_values = []

# Create trackbars
for i in range(0, face_components):
    tracker_values.append(max_tracker_value//2)
    cv2.createTrackbar("Weights" + str(i), "Trackbars", max_tracker_value//2, max_tracker_value, create_new_face)
    
# Reset tracker by clicking on the mean image
cv2.setMouseCallback("Result", reset_tracker_values)

print('''Usage: 
        Change the weights using the trackers
        Click on the "Result" window to reset trackers
        Hit ESC to terminate program.''')

cv2.waitKey(0)
cv2.destroyAllWindows()

Reading images from images/celeba_faces...
237 files read.
The size of the image is (218, 178, 3)
Creating data matrix...DONE!
The shape of the data matrix is (474, 116412)
Calculating PCA ...
The shape of the eigen_vectors is (10, 116412)
PCA compute is done!
The shape of the output (436, 356, 3)
Usage: 
        Change the weights using the trackers
        Click on the "Result" window to reset trackers
        Hit ESC to terminate program.
